In [111]:
pip install gradio


Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 23.1.2 -> 23.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [112]:
pip install scikit-image

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 23.1.2 -> 23.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [2]:
import numpy as np
import matplotlib.pyplot as plt
from skimage import transform
import gradio as gr
import torch
import torchvision.transforms as transforms
from PIL import Image


In [3]:
import gradio as gr
from skimage import  transform

def load_segmentation_model():
    model = torch.load("model.pth")
    model.eval()
    return model

def process_masks2(masks_array):
    new_masks_array = np.where(masks_array > 0, 1, 0)
    return new_masks_array

def preprocess(img):
    img = img.convert("RGB")
    return img

def image_classifier(img):
    

    frame_array = np.array(img)

    frame_array  = np.array(Image.fromarray(frame_array ).resize((256,256)))
    frame = frame_array 
    
    
    frame_array =  frame_array [np.newaxis, ...]
    
    frame_tensor = torch.tensor(frame_array, dtype=torch.float32).permute(0, 3, 1, 2)
    frame_tensor /= 255.0
 
    
    model = load_segmentation_model()
    
    with torch.no_grad():
        prediction = model(frame_tensor)
    
    prediction = prediction.cpu().numpy()
    prediction = process_masks2(prediction)
    prediction = np.squeeze(prediction)
    prediction = np.expand_dims(prediction , axis=-1)
    prediction = prediction * frame
    image = prediction.astype(np.uint8)
    #image = np.squeeze(image)
    image = Image.fromarray(image)
    return image
    
demo = gr.Interface(fn=image_classifier, inputs="image", outputs="image")
demo.launch()


Running on local URL:  http://127.0.0.1:7861

To create a public link, set `share=True` in `launch()`.
